In [ ]:
import pandas as pd
df = pd.read_excel("/Users/macos/Desktop/Edison-ai/data/ner/charbased/oesm18nat/national_M2018_dl.xlsx")
print('ok')

In [6]:
df.info()
print('ok')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1379 entries, 0 to 1378
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   OCC_CODE   1379 non-null   object 
 1   OCC_TITLE  1379 non-null   object 
 2   OCC_GROUP  1379 non-null   object 
 3   TOT_EMP    1379 non-null   int64  
 4   EMP_PRSE   1379 non-null   float64
 5   H_MEAN     1379 non-null   object 
 6   A_MEAN     1379 non-null   object 
 7   MEAN_PRSE  1379 non-null   float64
 8   H_PCT10    1379 non-null   object 
 9   H_PCT25    1379 non-null   object 
 10  H_MEDIAN   1379 non-null   object 
 11  H_PCT75    1379 non-null   object 
 12  H_PCT90    1379 non-null   object 
 13  A_PCT10    1379 non-null   object 
 14  A_PCT25    1379 non-null   object 
 15  A_MEDIAN   1379 non-null   object 
 16  A_PCT75    1379 non-null   object 
 17  A_PCT90    1379 non-null   object 
 18  ANNUAL     82 non-null     object 
 19  HOURLY     6 non-null      object 
dtypes: float

In [3]:
occ = df['OCC_TITLE']
print(occ)

0                             All Occupations
1                      Management Occupations
2                              Top Executives
3                            Chief Executives
4                            Chief Executives
                        ...                  
1374               Mine Shuttle Car Operators
1375        Tank Car, Truck, and Ship Loaders
1376        Tank Car, Truck, and Ship Loaders
1377    Miscellaneous Material Moving Workers
1378       Material Moving Workers, All Other
Name: OCC_TITLE, Length: 1379, dtype: object


In [8]:
import inflect
inflect = inflect.engine()

tit_occ = "/Users/macos/thunder/ner-tf/cls_data/tit_occ.txt"

lines_seen = set()
with open (tit_occ, 'w') as w:
    # def write_out(line):
    for num,line in enumerate(occ):
        if num>3:
        # tit = ""
            if line not in lines_seen:# and line != '\n':
                lines_seen.add(line)
                if inflect.singular_noun(line)==False:
                    # line1 = line[:-1]
                    w.writelines(line)
                    w.writelines('\n')
                else:
                    line = inflect.singular_noun(line)
                    w.writelines(line)
                    w.writelines('\n')
print('ok')

ok


In [19]:
## extract OESM18 Occupation data
lines_seen = set()
with open (tit_occ, 'w') as w:
    def write_out(line):
            if line not in lines_seen:# and line != '\n':
                lines_seen.add(line)
                if inflect.singular_noun(line)==False:
                    # line1 = line[:-1]
                    w.writelines(line)
                    w.writelines('\n')
                else:
                    line = inflect.singular_noun(line)
                    w.writelines(line)
                    w.writelines('\n')
    for num,line in enumerate(occ):
        tit = ""
        if num>3:
            doc = line.split()
            if 'and' in doc:
                n = doc.index('and')
                # print(n)
                tit = ''
                for i,word in enumerate(doc):
                    if word.endswith(','): #or doc[i+1]!='and':
                        tit = word[:-1] + ' ' + doc[-1]
                        # print('1' + tit)
                        write_out(tit)
                        tit = ''
                    elif word==('and') and (doc[i-1].endswith(',')==False):
                        tit+=doc[-1]
                        # print('2' + tit)
                        write_out(tit)
                        tit = ''
                        continue
                    elif word==('and') and (doc[i-1].endswith(',')==True):
                        tit+=doc[-1]
                        # print('2' + tit)
                        write_out(tit)
                        tit = ''
                        continue
                    elif word == doc[-1]:
                        tit+= word
                        # print('3' + tit)
                        write_out(tit)
                        continue
                    else:
                        tit+= word + ' '
                        continue
            else:
                write_out(line) 
                    
#manually remove excepts, others
print('ok')

ok


In [27]:
## CONCAT
file1 = "/Users/macos/thunder/ner-tf/cls_data/title.txt"
file2 = "/Users/macos/thunder/ner-tf/cls_data/tit_occ.txt"
combined = "/Users/macos/thunder/ner-tf/cls_data/tit.txt"

import os, glob
files = glob.glob('/Users/macos/thunder/ner-tf/cls_data/tit*')

all_lines = []
for f in files:
    print(f)
    with open(f,'r') as fi:
        all_lines += fi.readlines()
all_lines = set(all_lines)

with open(combined,'w') as fo:
    fo.write("".join(all_lines))

/Users/macos/thunder/ner-tf/cls_data/tit_occ.txt
/Users/macos/thunder/ner-tf/cls_data/title.txt


In [17]:
## split us street names without designation
str_file = '/Users/macos/thunder/ner-tf/cls_data/extra/street_name.txt'
str_only = '/Users/macos/thunder/ner-tf/jeff_work/place_list/str_only.txt'

str_list = []
with open (str_file, 'r') as r:
    for line in r.readlines():
        str_list.append(' '.join(line.split(' ')[:-1]).title())
        # remove last word
        
with open (str_only, 'w') as w:
    for street in str_list:
        w.writelines(street)
        w.writelines('\n')
print('ok')

ok


In [7]:
# Extract US cities, states
print('start')
import pandas as pd
from tqdm import tqdm

city_file = '/Users/macos/thunder/ner-tf/jeff_work/place_list/city.txt'
state_file = '/Users/macos/thunder/ner-tf/jeff_work/place_list/state.txt'
us_city = '/Users/macos/Desktop/Edison-ai/data/ner/loc/simplemaps_uscities_basicv1.6/uscities.csv'


pd.set_option('display.max_rows', 1000)
df = pd.read_csv(us_city)
# df.head()
body = (df['city'])
states = df['state_name']
with open(city_file, 'w') as w:
    for city in body:
        w.writelines(city)
        w.writelines('\n')
with open(state_file, 'w') as v:
    for state in states:
        v.writelines(state)
        v.writelines('\n')
print('ok')

start


914


In [3]:
outfilename = '/Users/macos/thunder/ner-tf/cls_data/tel_clean.txt'
infilename ='/Users/macos/Desktop/nc/data/train/tel.txt'
lines_seen = set() # holds lines already seen
outfile = open(outfilename, "w")
for line in open(infilename, "r"):
    if line not in lines_seen: # not a duplicate
        outfile.write(line)
        lines_seen.add(line)
outfile.close()
print('ok')

ok


In [16]:
# Extract occupations CSV data from oneTonline
print('start')
import pandas as pd

occ_out = '/Users/macos/thunder/ner-tf/cls_data/extra/occ.txt'
occ_file = '/Users/macos/Desktop/Edison-ai/data/ner/occupations/All_Occupations.csv'


pd.set_option('display.max_rows', 1000)
df = pd.read_csv(occ_file)
# df.head()
body = (df['Occupation'])
with open(occ_out, 'w') as w:
    for occ in body:
        w.writelines(occ)
        w.writelines('\n')

start


In [3]:
## extract job_skills from google
print('start')
import pandas as pd
from tqdm import tqdm

jobs_out = '/Users/macos/thunder/ner-tf/cls_data/extra/jobs.txt'
jobs_file = '/Users/macos/Desktop/Edison-ai/data/ner/occupations/job_skills.csv'


pd.set_option('display.max_rows', 1000)
df = pd.read_csv(jobs_file)
# df.head()
body = (df['Title'])
info = (df['Category'])
with open(jobs_out, 'w') as w:
    for tit in body:
        w.writelines(tit)
        w.writelines('\n')
    for cat in tqdm(info):
        w.writelines(cat)
        w.writelines('\n')
print('ok')

100%|██████████| 1250/1250 [00:00<00:00, 196753.11it/s]


start
ok


In [ ]:
## extract job_skills peopledatalabs
print('start')
import pandas as pd
from tqdm import tqdm
from googletrans import Translator

jobs_out = '/Users/macos/thunder/ner-tf/cls_data/extra/title_skill.txt'
jobs_file = '/Users/macos/Desktop/Edison-ai/data/ner/occupations/pdl_related_title_dataset/2019_free_title_data.csv'


translator = Translator()
pd.set_option('display.max_rows', 1000)
df = pd.read_csv(jobs_file)
df.head()
body = (df['title'])
info = (df['top related titles'])
with open(jobs_out, 'w') as w:
        for n,tit in enumerate(tqdm(body)):
            try:
                trans = translator.translate(tit)
                w.writelines(trans.text)
                w.writelines('\n')
            except:
                try:
                     w.writelines(tit)
                     w.writelines('\n')
                except:
                    pass
             # print(n)
        for m,rel in enumerate(tqdm(info)):
            try:
                trans1 = translator.translate(rel)
                w.writelines(trans1.text)
                w.writelines('\n')
            except:
                try:
                    w.writelines(rel)
                    w.writelines('\n')
                except:
                    pass
print('ok')

start


 66%|██████▌   | 3276/5000 [23:22<12:15,  2.34it/s]

In [2]:
## extract sf street names
print('start')
import pandas as pd
name_file ='/Users/macos/Desktop/Edison-ai/data/ner/loc/sf-street-names/street-names.csv'
nat_out = '/Users/macos/thunder/ner-tf/cls_data/extra/street_name.txt'

pd.set_option('display.max_rows', 1000)
df = pd.read_csv(name_file)
df.head()
body = (df['FullStreetName'])
with open(nat_out, 'w') as w:
    for occ in body:
        w.writelines(occ)
        w.writelines('\n')
print('ok')

start
ok


In [15]:
## extract QA
import pandas as pd
from tqdm import tqdm

print('start!')
qa_file = '/Users/macos/Desktop/Edison-ai/ner-tflite/qa_test/SigWeb/0327/fail_info.xls'
# baby_file = '/Users/macos/Desktop/Edison-ai/data/ner/names/us-baby-names/StateNames.csv'
qa_out = '/Users/macos/thunder/ner-tf/cls_data/extra/qa.txt'

pd.set_option('display.max_rows', 1000)
df = pd.read_excel(qa_file)
# df.head()
with open(qa_out, 'w') as w:
    # body = (df['STREET'])
    names = (df['exp'])
    try:
        for name in tqdm(names):
            # print(occ)
            w.writelines(name.split())
            w.writelines('\n')
    except:
        print('error!')
        pass


100%|██████████| 86/86 [00:00<00:00, 30550.53it/s]


start!


In [ ]:
import pandas as pd
from tqdm import tqdm

print('start!')
baby_file = '/Users/macos/Desktop/Edison-ai/data/ner/names/us-baby-names/NationalNames.csv'
# baby_file = '/Users/macos/Desktop/Edison-ai/data/ner/names/us-baby-names/StateNames.csv'
baby_out = '/Users/macos/thunder/ner-tf/cls_data/extra/baby_name.txt'

pd.set_option('display.max_rows', 1000)
df = pd.read_csv(baby_file)
# df.head()
with open(baby_out, 'w') as w:
    # body = (df['STREET'])
    names = (df['Name'])
    try:
        for name in tqdm(names):
            # print(occ)
            w.writelines(name)
            w.writelines('\n')
    except:
        print('error!')
        pass

In [ ]:
## extract la street names
print('start')
import pandas as pd
la_file ='/Users/macos/Desktop/Edison-ai/data/ner/loc/openaddresses-north-america-excluding-us/mexico.csv'
la_out = '/Users/macos/thunder/ner-tf/cls_data/extra/mex_name.txt'

pd.set_option('display.max_rows', 1000)
df = pd.read_csv(la_file)
# df.head()
with open(la_out, 'w') as w:
    body = (df['STREET'])
    city = (df['CITY'])
    try:
        for occ in tqdm(body):
            # print(occ)
            w.writelines(occ)
            w.writelines('\n')
    except:
        pass
    try:
        for cit in city:
            w.writelines(cit)
            w.writelines('\n')
    except:
        pass
print('ok')

In [16]:
## extract chicago street names
print('start')
import pandas as pd
ch_file ='/Users/macos/Desktop/Edison-ai/data/ner/loc/chicago-street-names/chicago-street-names.csv'
ch_out = '/Users/macos/thunder/ner-tf/cls_data/extra/chic_name.txt'

pd.set_option('display.max_rows', 1000)
df = pd.read_csv(ch_file)
df.head()
body = (df['Full Street Name'])
with open(ch_out, 'w') as w:
    for occ in body:
        occ = occ[2:]
        w.writelines(occ)
        w.writelines('\n')
print('ok')

start
ok


In [4]:
## extract us ne street names
print('start!')
import pandas as pd
from tqdm import tqdm
ch_dir ='/Users/macos/Desktop/Edison-ai/data/ner/loc/openaddresses-us-northeast/'
south_dir = '/Users/macos/Desktop/Edison-ai/data/ner/loc/openaddresses-us-south/'
ch_out = '/Users/macos/thunder/ner-tf/cls_data/extra/us_name.txt'
mw_dir = '/Users/macos/Desktop/Edison-ai/data/ner/loc/openaddresses-us-west/'

import os
filelist = os.listdir(mw_dir)
with open(ch_out, 'w') as w:
    for i in tqdm(filelist):
        if i.endswith(".csv"):  # You could also add "and i.startswith('f'
            df = pd.read_csv(mw_dir + i)
            body = (df['STREET'])
            city = (df['CITY'])
            try:
                for occ in body:
                    # print(occ)
                    w.writelines(occ)
                    w.writelines('\n')
            except:
                pass
            try:
                for cit in city:
                    w.writelines(cit)
                    w.writelines('\n')
            except:
                pass
print('ok')

  8%|▊         | 1/13 [00:06<01:14,  6.19s/it]/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 15%|█▌        | 2/13 [00:13<01:12,  6.60s/it]/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 38%|███▊      | 5/13 [00:21<00:31,  3.89s/it]/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3,4,5,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 54%|█████▍    | 7/13 [00:55<00:55,  9.22s/it]/usr/local/lib/python3.7/site-packages/IPython/core/inter

start!
ok


In [5]:
##split by 2 lines
hold_file = '/Users/macos/thunder/ner-tf/jeff_work/process_data/hold.txt'
output_file = '/Users/macos/thunder/ner-tf/cls_data/clean.txt'

with open(hold_file, "r") as r, open(output_file, "w") as w:
    z = r.readlines()
    twol = range(0,len(z),2)
    for num, line in enumerate(z):
        if num in twol:
            loc = ' '.join(z[num:num+2])
            w.writelines(loc)
            w.writelines('\n')
            

In [1]:
print('start!')
from tqdm import tqdm

## Remove names from ORG
name_list = ['bob', 'bob lim', 'jeff']
lines = """CLAWI - Mundo Gaturro
CityOdds
BettingVille
Ototespit . com
iLogistix
MotorExchange
Market Research
Fingerworks
Ladenburg Thalmann
Encryptosoft
PolySpot Enterprise Search
bob works
bob
TrenMedia
jeffur
JadeTrack"""
name_file = '/Users/macos/Desktop/nc/data/train/name.txt'
org_file  = '/Users/macos/Desktop/nc/data/train/org.txt'
new_org = '/Users/macos/thunder/ner-tf/jeff_work/new_org.txt'

body = []
name_list = []
with open(name_file, 'r') as r, open(org_file, 'r') as s, open(new_org, 'w') as w:
    name_body = r.readlines()
    for name in tqdm(name_body):
        name_list.append(name)
        
    org_body = s.readlines()
    for org in tqdm(org_body):
        body.append(org)
    # body = org_body.split('\n')
    print('time to check!')
# body = lines.split('\n')
# [i for i in name_list if i in body]
# print(i)
    for i in tqdm(name_list):
        if i in body:
            body.remove(i)
            # print(i)
        # else:
        #     w.writelines(i)
        #     w.writelines('\n')
    for ele in body:
        w.writelines(ele)
        # w.writelines('\n')
print('ok')
# for line in lines:
#     if name_list in :
#         print(line)
#     else:
#         print('nope!')
        

100%|██████████| 173965/173965 [19:26<00:00, 149.17it/s]


start!
time to check!
ok


In [1]:
from tqdm import tqdm

input_file = "/Users/macos/thunder/ner-tf/jeff_work/process_data/hold.txt"
body = []
with open(input_file, "r") as fp:
    source = fp.readlines()
    for line in source:
        newline = line.replace(',', '')
        body.append(newline)
    
output_file = "/Users/macos/thunder/ner-tf/cls_data/clean.txt"
with open(output_file, "w") as fp:
    for line in tqdm(body):
        fp.write(line)
print('ok')

100%|██████████| 29830/29830 [00:00<00:00, 864234.47it/s]


ok


In [4]:
## BEST IN REMOVING DUPLICATES, WHITE SPACE
## 
from tqdm import tqdm
import random

print('start')
input_file = "/Users/macos/thunder/ner-tf/jeff_work/process_data/hold.txt"
# input_File = "/Users/macos/thunder/ner-tf/cls_data/extra/company_us.txt"
with open(input_file, "r") as fp:
    source = fp.readlines()
    for i in range(2):   
        data = [ (random.random(), line) for line in source ]
        data.sort() 
    #shuffle lines
    
    new_lines = []
    for _,line in tqdm(data):
        #- Strip white spaces
        line = line.strip()
        if line not in new_lines:
            new_lines.append(line)

output_file = "/Users/macos/thunder/ner-tf/cls_data/clean.txt"
with open(output_file, "w") as fp:
    fp.write("\n".join(new_lines))
print('ok')


100%|██████████| 18395/18395 [00:03<00:00, 5916.92it/s]


start
ok
